<a href="https://colab.research.google.com/github/Bigulac/MOAR-Dense_network/blob/main/MOAR_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Potpuno povezana neuralna mreža

In [2]:
import os
import numpy as np
import keras
from keras import layers
from keras import ops
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

In [5]:
# Skida dataset sa interneta
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
!unzip -q kagglecatsanddogs_5340.zip

# Izbacuje loše slike
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Obrisano {num_skipped} slika.")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  79.0M      0  0:00:09  0:00:09 --:--:-- 90.1M
Obrisano 1590 slika.


In [6]:
image_size = (180, 180)
batch_size = 128

# Pravi skup slika za treninranje modela i validaciju
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
'''
# Prikaz slika iz dataseta
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title("Pas" if int(labels[i]) else "Mačka")
        plt.axis("off")
'''
# Povećanje skupa malim modifikacijama postojećih slika
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images
'''
# Prikaz kako izgledaju slike nakon modifikacija
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(augmented_images[0]).astype("uint8"))
        plt.axis("off")
'''
# Primena gornje funkcije na trening skup
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)

# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Using 4682 files for validation.


Odavde se razlikuje kod. Primer neuralne mreže sa potpuno povezanim slojevima. Primeri sa različitom dubinom i širinom i uporedni prikaz efikasnosti/pouzdanosti.

In [8]:
inputs = keras.Input(shape=image_size + (3,))

# inputs.shape

x = layers.Rescaling(1.0 / 255)(inputs)
x = layers.Flatten()(x)
'''
dense = layers.Dense(64, activation="relu")
x = dense(inputs)'''

x = layers.Dense(1024, activation="relu")(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)

outputs = layers.Dense(1, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="Dense_model")

model.summary()

# keras.utils.plot_model(model, show_shapes=True)


Model: "Dense_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling_1 (Rescaling)              │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 97200)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │      99,533,824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 100,239,489 (382.38 MB)

 Trainable params: 100,239,489 (382.38 MB)

 Non-trainable params: 0 (0.00 B)

**Treniranje modela**

In [10]:
epochs = 10

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)


Epoch 1/10


KeyboardInterrupt: 

Grafik zafisnosti preciznosti od broja epoha

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Preciznost modela')
plt.ylabel('preciznost[%]')
plt.xlabel('epoha')
# plt.legend(['training'], loc='best')
plt.show()

# Napraviti da pravljenje modela bude funkcija kako bi mogao da napraviš
# više modela sa različitim brojem epoha %%